# ChatGPT
Здесь построим скрипт для вывода оценки отзывов пользователей. На вход получаем .csv таблицу с четырьмя колонками (email review text date rate), последнюю нужно заполнить оценкой. Затем сортируем и сохраняем новую таблицу

In [74]:
openai.api_key = '############' # ключ от openAI
FILENAME = 'filename' # '{FILENAME}_analyzed.csv' - название нового фрейма

In [1]:
### устанавливаем при необходимости
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.1/70.1 kB 566.8 kB/s eta 0:00:001m363.0 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 3.1 MB/s eta 0:00:002.7 MB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 11.3 MB/s eta 0:00:00


## Разведка

In [42]:
import pandas as pd
import os
import openai

In [90]:
# проверяем запрос на примере
question = """
Rate the review as a number to the nearest thousandth from 1 to 10 from the most negative to the most enthusiastic. Answer with one number.
Review:
I found Welltory to be a bit of a letdown. While the app provides some interesting data about my health, I didn't find it particularly useful in terms of making meaningful changes in my life. The guided meditations and breathing exercises were okay, but I didn't find them to be any better than other meditation apps out there.
"""
response = openai.Completion.create(model="text-davinci-003", prompt=question, temperature=0, max_tokens=7)
response

<OpenAIObject text_completion id=cmpl-6u9OoLj8ZMnsNkzkKevmTzmZCkKWU at 0x7f75103adc70> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "text": "\n3.50"
    }
  ],
  "created": 1678840518,
  "id": "cmpl-6u9OoLj8ZMnsNkzkKevmTzmZCkKWU",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 4,
    "prompt_tokens": 106,
    "total_tokens": 110
  }
}

In [39]:
# парсим вывод
float(response['choices'][0]['text'][1:])

3.5

In [82]:
### Загрузка по ссылке
# SHEET_NAME = 'Data'
# SHEET_ID = '1NEdh5NkMJPRHgXxLryKGv20NmBZidEBCfJatdIZsTBo'
# url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
# # url = https://docs.google.com/spreadsheets/d/1RpHXQgM1_1zIrLdbjM7HxpgfLNvDBWrqTZSeHd7CikA/edit#gid=0"
# df = pd.read_csv(url)
# df.head()

### Simple Download
df = pd.read_csv('Welltory Test_Python Developer_App Reviews - Data.csv')
df

,email,review text,date,rate
0,bryantjames@example.com,As someone who has struggled with anxiety and ...,2023-01-15,NaN
1,lgreen@example.net,Welltory is a decent app overall. I appreciate...,2023-02-01,NaN
2,ryanryan@example.com,I like using Welltory to track my stress level...,2023-01-02,NaN
3,spearsjamie@example.org,I was really disappointed with Welltory. The H...,2023-02-05,NaN
4,jeffreyvincent@example.org,"I've been using Welltory for a few months now,...",2023-02-03,NaN
5,kevin03@example.com,"Welltory is okay, but I'm not sure it's worth ...",2023-01-12,NaN
6,williamwang@example.org,"I've been using Welltory for a few weeks now, ...",2023-01-28,NaN
7,robertbautista@example.net,"Welltory is a decent app, but I find the HRV m...",2023-01-20,NaN
8,alexandermarshall@example.net,As someone who has struggled with anxiety and ...,2023-01-03,NaN
9,josephtucker@example.net,I found Welltory to be a bit of a letdown. Whi...,2023-02-04,NaN


In [83]:
base_question = """
Rate the review as a number to the nearest thousandth from 1 to 10 from the most negative to the most enthusiastic. Answer with one number.
Review:
"""
df2 = df.copy()

for i in range(len(df)):
    add_to_question = df['review text'][i]
    question = base_question + add_to_question
    response = openai.Completion.create(model="text-davinci-003", prompt=question, temperature=0, max_tokens=7)
    answer = float(response['choices'][0]['text'][1:])
    df2['rate'][i] = answer
    
df2

/tmp/ipykernel_18801/1607784420.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['rate'][i] = answer


,email,review text,date,rate
0,bryantjames@example.com,As someone who has struggled with anxiety and ...,2023-01-15,9.98
1,lgreen@example.net,Welltory is a decent app overall. I appreciate...,2023-02-01,7.33
2,ryanryan@example.com,I like using Welltory to track my stress level...,2023-01-02,7.33
3,spearsjamie@example.org,I was really disappointed with Welltory. The H...,2023-02-05,2.00
4,jeffreyvincent@example.org,"I've been using Welltory for a few months now,...",2023-02-03,9.98
5,kevin03@example.com,"Welltory is okay, but I'm not sure it's worth ...",2023-01-12,5.50
6,williamwang@example.org,"I've been using Welltory for a few weeks now, ...",2023-01-28,7.33
7,robertbautista@example.net,"Welltory is a decent app, but I find the HRV m...",2023-01-20,6.33
8,alexandermarshall@example.net,As someone who has struggled with anxiety and ...,2023-01-03,9.98
9,josephtucker@example.net,I found Welltory to be a bit of a letdown. Whi...,2023-02-04,3.50


In [ ]:
# df2.sort_values(['rate'], inplace=True, ascending=False)
df2

In [81]:
df2.to_csv('{}_analyzed.csv'.format(filename))

## Итоговый скрипт

In [5]:
import pandas as pd
import os
import openai

openai.api_key = '################FILENAME = 'filename'

df = pd.read_csv('Welltory Test_Python Developer_App Reviews - Data.csv')

base_question = """
Rate the feedback as a number to the nearest thousandth from 1. to 10. from the most negative to the most enthusiastic. Answer with one number.
feedback:
"""

# df2 = df.copy()

for i in range(len(df)):
    add_to_question = df['review text'][i]
    question = base_question + add_to_question
    response = openai.Completion.create(model="text-davinci-003", prompt=question, temperature=0, max_tokens=7)
    answer = float(response['choices'][0]['text'][1:])
    df['rate'][i] = answer
    
df.sort_values(['rate'], inplace=True, ascending=False)
df.to_csv('{}_analyzed.csv'.format(FILENAME))

/tmp/ipykernel_26278/2768785337.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['rate'][i] = answer


## Прочее

In [10]:
question = """
Напиши ответ:
Создайте python-скрипт, который с помощью ChatGPT сможет оценивать отзывы из таблицы по колонке 'review text', используя OpenAI API. Скрипт должен читать файл из директории, проставлять оценку в колонку 'rate', сортировать отзывы по убыванию оценки, записывать в новый файл с названием вида “%filename%_analyzed.csv” и складывать в ту же директорию.
Файл запроса таблицы .csv с колонками ['email', 'review text', 'date', 'rate']
"""
# response = openai.Completion.create(model="text-davinci-003", prompt=question, temperature=0, max_tokens=7)
response = openai.Completion.create(model="text-davinci-003", prompt=question, temperature=0,)
response

<OpenAIObject text_completion id=cmpl-6u9nr0oEzYh2umSV0Pqrz0iS5pQc0 at 0x7fd2df3d0950> JSON: {
  "choices": [
    {
      "finish_reason": "length",
      "index": 0,
      "logprobs": null,
      "text": "\nimport pandas as pd\nfrom openai import API\nfrom open"
    }
  ],
  "created": 1678842071,
  "id": "cmpl-6u9nr0oEzYh2umSV0Pqrz0iS5pQc0",
  "model": "text-davinci-003",
  "object": "text_completion",
  "usage": {
    "completion_tokens": 16,
    "prompt_tokens": 402,
    "total_tokens": 418
  }
}

In [11]:
df

,email,review text,date,rate
0,bryantjames@example.com,As someone who has struggled with anxiety and ...,2023-01-15,10.000
4,jeffreyvincent@example.org,"I've been using Welltory for a few months now,...",2023-02-03,10.000
8,alexandermarshall@example.net,As someone who has struggled with anxiety and ...,2023-01-03,10.000
1,lgreen@example.net,Welltory is a decent app overall. I appreciate...,2023-02-01,7.333
2,ryanryan@example.com,I like using Welltory to track my stress level...,2023-01-02,7.333
6,williamwang@example.org,"I've been using Welltory for a few weeks now, ...",2023-01-28,7.333
7,robertbautista@example.net,"Welltory is a decent app, but I find the HRV m...",2023-01-20,6.333
5,kevin03@example.com,"Welltory is okay, but I'm not sure it's worth ...",2023-01-12,5.333
3,spearsjamie@example.org,I was really disappointed with Welltory. The H...,2023-02-05,2.000
9,josephtucker@example.net,I found Welltory to be a bit of a letdown. Whi...,2023-02-04,2.000
